In [1]:
import json
import os
import ast
import csv
import io
from io import StringIO, BytesIO, TextIOWrapper
import gzip
from datetime import datetime, date
from s3_utils import *
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import ast
from datetime import timedelta
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.model_selection import cross_val_score, ShuffleSplit, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import warnings
import sys
import time
import torch
from statsmodels.tsa.stattools import adfuller, acf
from utils import *
from src.stationary_test_utils import *

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-r2ty3snk because the default path (/home/ubuntu/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:

kalman_df = pd.read_csv('./imputed_df_sleep_fatigue/kalman_impute_df_userid_1032.csv')

feature_name = kalman_df.columns.to_list()
feature_name = ['answer_text', 'hr_average', 'rmssd']
feature_name = list(filter(lambda item: "user" not in item.lower() and "date" not in item.lower() and "X" != item, feature_name))
print(feature_name)

# window_select(kalman_df, feature_name, ADFTestResultRetriver(kalman_df, feature_name))
# window_select(kalman_df, feature_name, ADFSingleWindowSelector(kalman_df, feature_name))
window_select(kalman_df, feature_name, ACFSingleWindowSelector(kalman_df, feature_name, method='avg'), minimal_window_size=5)


['answer_text', 'hr_average', 'rmssd']
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671, 0.7598])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671, 0.7598, 0.7333])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671, 0.7598, 0.7333, 0.7219])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671, 0.7598, 0.7333, 0.7219,
        0.7152])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671, 0.7598, 0.7333, 0.7219,
        0.7152, 0.6966])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671, 0.7598, 0.7333, 0.7219,
        0.7152, 0.6966, 0.6846])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671, 0.7598, 0.7333, 0.7219,
        0.7152, 0.6966, 0.6846, 0.6707])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671, 0.7598, 0.7333, 0.7219,
        0.7152, 0.6966, 0.6846, 0.6707, 0.6576])
tensor([1.0000, 0.8726, 0.8414, 0.8241, 0.8087, 0.7671, 0.7598, 0.7333,

[0,
 17,
 34,
 52,
 70,
 88,
 105,
 120,
 134,
 146,
 154,
 160,
 166,
 172,
 177,
 182,
 187,
 192,
 197,
 202,
 207,
 212,
 217,
 222,
 227,
 232,
 237,
 242,
 247,
 252,
 257,
 262,
 267]

False


In [20]:

# def window_select_acf(df, features):
#     acfs = []
#     for feature in features:
#         series = df[feature]
#         acfs.append(torch.abs(torch.Tensor(acf(series, nlags=len(series) - 1))))
#     print(acfs)
# window_select_acf(kalman_df, feature_name)
def get_acf_list(df, features, start_window, curr_window_size):
    acf_value_list = []
    for feature in features:
        series = df[start_window:][feature]
        acf_values = torch.abs(torch.Tensor(acf(series, nlags=len(series) - 1)))
        print(acf_values)
        
        acf_value_list.append(acf_values[:curr_window_size])
    acf_value_list = torch.stack(acf_value_list)
    print(acf_value_list.shape)
    print(torch.min(acf_value_list, axis=0)[0])
    return acf_value_list
# def window_select(df, features, minimal_window_size=7, get_value_list=get_acf_list, constraint_test=):
#     curr_window_size = minimal_window_size
#     total_len = len(df)
#     start_window = 0
#     p_value_list = []
#     break_points = []
#     while start_window + curr_window_size < total_len:
#         p_value_list = get_value_list(df, features, start_window, curr_window_size)
#         constraint_satisfied = constraint_test(p_value_list, "avg")
#         if not constraint_satisfied:
#             break_points.append(start_window)
#             start_window = start_window + curr_window_size
#             curr_window_size = minimal_window_size
#         else:
#             curr_window_size += 1
#     return break_points
get_acf_list(kalman_df, ['hr_average', 'rmssd'], 0, 8)

tensor([1.0000e+00, 8.5175e-01, 8.1481e-01, 7.9606e-01, 7.8889e-01, 7.3159e-01,
        7.2643e-01, 6.8678e-01, 6.5974e-01, 6.5286e-01, 6.4287e-01, 6.2317e-01,
        6.0399e-01, 5.9053e-01, 5.4583e-01, 5.5083e-01, 5.2000e-01, 5.0247e-01,
        4.6967e-01, 4.6307e-01, 4.4055e-01, 4.4246e-01, 4.4823e-01, 4.3267e-01,
        4.1963e-01, 4.0559e-01, 3.9653e-01, 3.9212e-01, 3.9727e-01, 3.7030e-01,
        3.7321e-01, 3.5807e-01, 3.4900e-01, 3.1757e-01, 3.2717e-01, 3.0114e-01,
        2.6316e-01, 2.5477e-01, 2.2896e-01, 1.9166e-01, 1.5985e-01, 1.5403e-01,
        1.3093e-01, 1.2265e-01, 1.0882e-01, 1.0852e-01, 7.1664e-02, 5.7222e-02,
        3.3970e-02, 1.3026e-02, 1.2210e-02, 2.1719e-02, 3.0477e-02, 5.3914e-02,
        7.1800e-02, 7.7969e-02, 8.7914e-02, 1.0586e-01, 1.0597e-01, 1.1436e-01,
        1.2249e-01, 1.3623e-01, 1.4587e-01, 1.6690e-01, 1.7410e-01, 1.7899e-01,
        2.0328e-01, 2.0536e-01, 2.0741e-01, 2.1715e-01, 2.3208e-01, 2.3748e-01,
        2.4080e-01, 2.4407e-01, 2.5194e-

tensor([[1.0000, 0.8518, 0.8148, 0.7961, 0.7889, 0.7316, 0.7264, 0.6868],
        [1.0000, 0.8801, 0.8370, 0.8283, 0.7970, 0.7419, 0.7344, 0.6860]])

In [15]:
get_acf_list(kalman_df, ['hr_average'], 14, len(kalman_df)-14)

14    -0.622555
15    -0.726196
16     0.030387
17    -0.683259
18    -0.607749
         ...   
271   -0.011069
272   -0.692142
273   -1.879579
274   -0.973455
275   -1.071344
Name: hr_average, Length: 262, dtype: float64
tensor([1.0000e+00, 8.5731e-01, 8.1870e-01, 7.9719e-01, 7.8947e-01, 7.3282e-01,
        7.2565e-01, 6.8574e-01, 6.5756e-01, 6.4566e-01, 6.3891e-01, 6.1773e-01,
        5.9675e-01, 5.8161e-01, 5.3389e-01, 5.3890e-01, 5.0802e-01, 4.8543e-01,
        4.5364e-01, 4.4759e-01, 4.2414e-01, 4.2437e-01, 4.3630e-01, 4.2264e-01,
        4.1106e-01, 3.9776e-01, 3.9000e-01, 3.8320e-01, 3.9097e-01, 3.6813e-01,
        3.6749e-01, 3.5219e-01, 3.4618e-01, 3.1899e-01, 3.2575e-01, 3.0043e-01,
        2.6461e-01, 2.5845e-01, 2.2802e-01, 1.9082e-01, 1.5812e-01, 1.5255e-01,
        1.2167e-01, 1.1565e-01, 1.0191e-01, 1.0020e-01, 5.9692e-02, 4.5995e-02,
        2.2092e-02, 5.7109e-04, 2.5222e-02, 4.1685e-02, 5.1007e-02, 7.4761e-02,
        9.3351e-02, 1.0355e-01, 1.1124e-01, 1.3145e-01, 1.

[0.0005710927653126419]

In [31]:
def p_value_constraint_test(p_value_list, method="max"):
    if method == "max":
        max_p_value = max(p_value_list)
        print(p_value_list)
        return max_p_value <= 0.05
    if method == "avg":
        print(np.mean(p_value_list))
        return np.mean(p_value_list) <= 0.05

def get_p_value_list(df, features, start_window, curr_window_size)
    p_value_list = []
    for feature in features:
        series = df[start_window:start_window+curr_window_size][feature]
        p_value = adfuller(series, autolag='AIC')[1]
        p_value_list.append(p_value)
    return p_value_list
    
def window_select_adf(df, features):
    minimal_window_size = 7
    curr_window_size = minimal_window_size
    total_len = len(df)
    start_window = 0
    p_value_list = []
    break_points = []
    while start_window + curr_window_size < total_len:
        p_value_list = get_p_value_list(df, features, start_window, curr_window_size)
        constraint_satisfied = p_value_constraint_test(p_value_list, "avg")
        if not constraint_satisfied:
            break_points.append(start_window)
            start_window = start_window + curr_window_size
            curr_window_size = minimal_window_size
        else:
            curr_window_size += 1
    return break_points
window_select_adf(kalman_df, feature_name)

0.286598960297648
0.15402929498961349
0.2913767916824355
0.2524406503404044
0.40178521691834196
0.011192079549319773
0.012969620760699222
0.003351754032949019
0.5725544825891805
0.036827928578765726
0.29952833859586636
0.1332147126082648
0.057255779287808464
0.20654147278442192
0.4965005482187161
nan
0.5204762331464731
0.021695822526096923
0.34832242386670764
0.266926623476886
0.19151155630601224
0.04207608662146523
0.21165043384009558
0.2391273912853605
0.09550138296179624
0.36106412747877725
0.20155621831218634
0.4903157937909909
0.1995571806641004
0.19362458859964943
0.32359038537230117
0.27658826164246586
0.3340869324498264
0.13444619609614603
0.14713316515500327
0.6153007704759093
0.48542488677551265
0.46530968625125535
0.3588906408532644
0.282453791907525
0.22632991960784973
0.2331784517250295
0.2004707997491829
0.32134985791935555


[0,
 7,
 14,
 21,
 28,
 35,
 45,
 53,
 60,
 67,
 74,
 81,
 88,
 95,
 103,
 110,
 117,
 125,
 132,
 139,
 146,
 153,
 160,
 167,
 174,
 181,
 188,
 195,
 202,
 209,
 216,
 223,
 230,
 237,
 244,
 251,
 258,
 265]

In [6]:
mean_df = pd.read_csv('./imputed_df_sleep_fatigue/mean_impute_df_userid_1032.csv')

In [20]:
def stationary_test(series):
    result = adfuller(series, autolag='AIC')
    print(f'ADF Statistic: {result[0]}')
    print(f'n_lags: {result[1]}')
    print(f'p-value: {result[1]}')
    for key, value in result[4].items():
        print('Critial Values:')
        print(f'   {key}, {value}')


stationary_test(kalman_df[0:8]['answer_text'])


ADF Statistic: -0.5661920377856459
n_lags: 0.8784397304792497
p-value: 0.8784397304792497
Critial Values:
   1%, -3.454988209954765
Critial Values:
   5%, -2.8723857312734613
Critial Values:
   10%, -2.572549407997327


In [25]:
stationary_test(kalman_df['hr_average'])


ADF Statistic: -1.8575439563705518
n_lags: 0.35228960159955947
p-value: 0.35228960159955947
Critial Values:
   1%, -3.4548039258751206
Critial Values:
   5%, -2.872304928618605
Critial Values:
   10%, -2.5725063100137175


In [26]:
stationary_test(kalman_df['breath_average'])

ADF Statistic: -3.027035128956414
n_lags: 0.03244563713705504
p-value: 0.03244563713705504
Critial Values:
   1%, -3.4548957220044336
Critial Values:
   5%, -2.8723451788613157
Critial Values:
   10%, -2.572527778361272


In [27]:
stationary_test(kalman_df['rmssd'])


ADF Statistic: -1.5314649203273019
n_lags: 0.5178926730737616
p-value: 0.5178926730737616
Critial Values:
   1%, -3.4554613060274972
Critial Values:
   5%, -2.8725931472675046
Critial Values:
   10%, -2.5726600403359887


In [9]:
stationary_test(mean_df['answer_text'])
stationary_test(mean_df['rmssd'])
stationary_test(mean_df['breath_average'])
stationary_test(mean_df['hr_average'])

ADF Statistic: -1.4913584583172514
n_lags: 0.5377817724658496
p-value: 0.5377817724658496
Critial Values:
   1%, -3.4548957220044336
Critial Values:
   5%, -2.8723451788613157
Critial Values:
   10%, -2.572527778361272
ADF Statistic: -1.909395314682709
n_lags: 0.3276542487032547
p-value: 0.3276542487032547
Critial Values:
   1%, -3.455365238788105
Critial Values:
   5%, -2.8725510317187024
Critial Values:
   10%, -2.5726375763314966
ADF Statistic: -3.1196408211122173
n_lags: 0.025133413121681127
p-value: 0.025133413121681127
Critial Values:
   1%, -3.4548957220044336
Critial Values:
   5%, -2.8723451788613157
Critial Values:
   10%, -2.572527778361272
ADF Statistic: -2.5482957417772925
n_lags: 0.10416113432665464
p-value: 0.10416113432665464
Critial Values:
   1%, -3.4546223782586534
Critial Values:
   5%, -2.8722253212300277
Critial Values:
   10%, -2.5724638500216264


In [11]:
test_df = pd.read_csv('/domino/datasets/local/Bump/mice_user_42.csv')

In [ ]:
test_df

In [15]:
import torch
from torch.utils import data
import matplotlib.pyplot as plt
import argparse
import math
import seaborn as sns; sns.set()
import sys

import numpy as np
import pickle
import os
import random

from statsmodels.tsa.stattools import adfuller
class TNCDataset(data.Dataset):
    def __init__(self, x, mc_sample_size, window_size, augmentation, epsilon=3, state=None, adf=False):
        super(TNCDataset, self).__init__()
        self.time_series = x
        self.T = x.shape[-1]
        self.window_size = window_size
        self.sliding_gap = int(window_size*25.2)
        self.window_per_sample = (self.T-2*self.window_size)//self.sliding_gap
        self.mc_sample_size = mc_sample_size
        self.state = state
        self.augmentation = augmentation
        self.adf = adf
        if not self.adf:
            self.epsilon = epsilon
            self.delta = 5*window_size*epsilon

    def __len__(self):
        return len(self.time_series)*self.augmentation

    def __getitem__(self, ind):
        ind = ind%len(self.time_series)
        print(2*self.window_size)
        print(self.T-2*self.window_size)
        t = np.random.randint(2*self.window_size, self.T-2*self.window_size)
        
        x_t = self.time_series[ind][:,t-self.window_size//2:t+self.window_size//2]
        plt.savefig('./plots/%s_seasonal.png'%ind)
        X_close = self._find_neighours(self.time_series[ind], t)
        X_distant = self._find_non_neighours(self.time_series[ind], t)

        if self.state is None:
            y_t = -1
        else:
            y_t = torch.round(torch.mean(self.state[ind][t-self.window_size//2:t+self.window_size//2]))
        return x_t, X_close, X_distant, y_t

    def _find_neighours(self, x, t):
        T = self.time_series.shape[-1]
        if self.adf:
            gap = self.window_size
            corr = []
            for w_t in range(self.window_size,4*self.window_size, gap):
                try:
                    p_val = 0
                    for f in range(x.shape[-2]):
                        p = adfuller(np.array(x[f, max(0,t - w_t):min(x.shape[-1], t + w_t)].reshape(-1, )))[1]
                        p_val += 0.01 if math.isnan(p) else p
                    corr.append(p_val/x.shape[-2])
                except:
                    corr.append(0.6)
            self.epsilon = len(corr) if len(np.where(np.array(corr) >= 0.01)[0])==0 else (np.where(np.array(corr) >= 0.01)[0][0] + 1)
            self.delta = 5*self.epsilon*self.window_size

        ## Random from a Gaussian
        t_p = [int(t+np.random.randn()*self.epsilon*self.window_size) for _ in range(self.mc_sample_size)]
        t_p = [max(self.window_size//2+1,min(t_pp,T-self.window_size//2)) for t_pp in t_p]
        x_p = torch.stack([x[:, t_ind-self.window_size//2:t_ind+self.window_size//2] for t_ind in t_p])
        return x_p

    def _find_non_neighours(self, x, t):
        T = self.time_series.shape[-1]
        if t>T/2:
            t_n = np.random.randint(self.window_size//2, max((t - self.delta + 1), self.window_size//2+1), self.mc_sample_size)
        else:
            t_n = np.random.randint(min((t + self.delta), (T - self.window_size-1)), (T - self.window_size//2), self.mc_sample_size)
        x_n = torch.stack([x[:, t_ind-self.window_size//2:t_ind+self.window_size//2] for t_ind in t_n])

        if len(x_n)==0:
            rand_t = np.random.randint(0,self.window_size//5)
            if t > T / 2:
                x_n = x[:,rand_t:rand_t+self.window_size].unsqueeze(0)
            else:
                x_n = x[:, T - rand_t - self.window_size:T - rand_t].unsqueeze(0)
        return x_n

In [3]:
import pandas as pd
kalman_df = pd.read_csv('./imputed_df_sleep_fatigue/kalman_impute_df_userid_1032.csv')
kalman_df


,X,date,user_id_x,answer_text,hr_average,rmssd,breath_average,score,user_id_y,birth_date
0,0,2021-10-25,1032,0.524594,-0.148764,0.225188,4.683366,76.000000,1032,2022-05-25
1,1,2021-10-26,1032,1.039473,-0.252406,0.404343,3.218078,75.000000,1032,2022-05-25
2,2,2021-10-27,1032,0.524594,-0.518913,0.897018,2.078409,89.000000,1032,2022-05-25
3,3,2021-10-28,1032,1.039473,-0.082138,0.449131,2.078409,83.000000,1032,2022-05-25
4,4,2021-10-29,1032,0.524594,-0.128036,0.449131,1.427170,77.000000,1032,2022-05-25
...,...,...,...,...,...,...,...,...,...,...
271,271,2022-07-11,1032,1.039473,-0.011069,-0.043545,-0.038118,74.000000,1032,2022-05-25
272,272,2022-07-12,1032,1.039473,-0.692142,0.941807,-0.200928,88.000000,1032,2022-05-25
273,273,2022-07-13,1032,1.039473,-1.879579,1.792793,-1.177787,89.000000,1032,2022-05-25
274,274,2022-07-14,1032,1.039473,-0.973455,0.493920,-0.200928,90.000000,1032,2022-05-25


In [24]:
ds = kalman_df[['hr_average', 'rmssd']]
ds = ds.to_numpy()
ds = ds.T

In [23]:
mydataset = TNCDataset(ds, mc_sample_size=20, window_size=7, augmentation=1, adf=True)
mydataset[0]

14
262


IndexError: invalid index to scalar variable.